In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd
from datetime import datetime
import os
import base64
import urllib.request 
from PIL import Image 
import uuid

In [ ]:
now = datetime.now()
dt = now.strftime("%Y%m%d%H%M%S")

## Connect to Cassandra Cluster

In [ ]:
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(['192.168.10.53', '192.168.10.54', '192.168.10.22'], port=9042, auth_provider=auth_provider)
session = cluster.connect('softnix') #Keyspaces

In [ ]:
url = "https://vincentarelbundock.github.io/Rdatasets/csv/AER/CPSSW9298.csv"
df = pd.read_csv(url)
df['datetime'] = dt
df = df.astype(str)
df.head(1)

In [ ]:
cols = "CREATE TABLE softnix.annual_enterprise ("
for col in df.columns:
    cols += f"{col} text,"
cols += "PRIMARY KEY (datetime, rownames)"
cols += ") WITH CLUSTERING ORDER BY (rownames ASC);"

In [ ]:
session.execute(cols)

In [ ]:
df.to_csv(f"/tmp/{dt}.csv", sep="|", index=False)
os.system(f'''
            cqlsh 192.168.10.53 9042 \
            -u cassandra -p cassandra \
            -e "COPY softnix.annual_enterprise \
            FROM '/tmp/{dt}.csv' WITH DELIMITER='|' AND HEADER=TRUE"
        ''')
os.system(f"rm -f /tmp/{dt}.csv")

## Save File to Cassandra (BLOB)

In [ ]:
'''Create Table Image Type BLOB'''
session.execute("""
    CREATE TABLE IF NOT EXISTS softnix.raw (
        id varchar PRIMARY KEY,
        image blob
    ) WITH CLUSTERING ORDER BY (id ASC);
""")

In [ ]:
'''Get Image from URL'''
urllib.request.urlretrieve( 
  'https://static.wixstatic.com/media/e59907_6a7d2908e7e34bb693cd5086a9df6c87~mv2.png/v1/fill/w_560,h_574,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/Image-empty-state.png', 
   "vin.png") 
image = open('vin.png', 'rb')
image_read = image.read()
image_64_encode = base64.encodebytes(image_read)

In [ ]:
'''Insert to Table'''
sql = f"INSERT INTO softnix.raw (id, image) VALUES (?, ?)"
pStatement = session.prepare(strCQL)
session.execute(pStatement,[str(uuid.uuid4()),image_64_encode])

In [ ]:
'''Convert Base64 to Image'''
img = session.execute("SELECT * FROM softnix.raw WHERE id = '02fc3af9-f8ea-4fdd-85b5-48ae5be50ea5'").one()[1]
with open("/home/cassandra/robert.jpg", "wb") as fh:
    fh.write(base64.decodebytes(img))

In [ ]:
session.execute("SELECT * FROM softnix.raw").all()